# Epidemias
Objetivo: relacionar o período ocupado das filas com epidemias.

### 5.1 Fração de períodos ocupados que terminam ###

Em cada um dos casos
1. M/M/1: ρ = 0.5, λ = 1, μ = 2
2. M/M/1: ρ = 0.5, λ = 2, μ = 4
3. M/M/1: ρ = 2, λ = 4, μ = 2

avalie qual a fração de árvores finitas? obs.: obviamente, você precisará usar uma heurística para dizer se a  ́arvore ́e finita ou infinita, tendo em vista que o computador não suporta trabalhar com estruturas infinitas.

Em particular, relacione s = G(s)

### 5.2 O que apresentar no relatório?

Apresentar, no mínimo, o seguinte
* resultados com intervalo de confiança
* indicação de verificação de corretude (resultados analíticos, no caso da fração de períodos ocupados que terminam, via s = G(s), dentro do intervalo de confiança)

In [1]:
import numpy as np
from collections import Counter
from scipy.stats import t
from statistics import mean

In [2]:
def branch_poisson(n, lam):
    z = np.zeros(n + 1)
    z[0] = 1
    for i in range(1, n + 1):
        z[i] = np.sum(np.random.poisson(lam, int(z[i - 1])))
    return z

n = 500000

In [3]:
simlist = np.array([branch_poisson(10, 0.5)[10] for _ in range(n)])
print(np.sum(simlist == 0) / n)

0.999338


In [4]:
simlist = np.array([branch_poisson(10, 0.5)[10] for _ in range(n)])
print(np.sum(simlist == 0) / n)

0.999388


In [5]:
simlist = np.array([branch_poisson(10, 2)[10] for _ in range(n)])
print(np.sum(simlist == 0) / n)

0.202498


In [211]:
def simulador(lambda_=1, mi_=2, simulation_total_time=10, deterministic=False):
    n, tempo_simulacao, num_arvores_finitas = 0, 0, 0
    exponential = (lambda mi: 1 / mi) if deterministic else (lambda mi: np.random.exponential(scale=1 / mi))

    while tempo_simulacao < simulation_total_time:
        tempo_chegada = np.random.exponential(scale=1 / lambda_)
        tempo_saida = exponential(mi_)

        if n == 0 or tempo_chegada < tempo_saida:
            tempo_simulacao += tempo_chegada
            n += 1
        else:
            tempo_simulacao += tempo_saida
            n -= 1
            if (n==0):
                num_arvores_finitas += 1

    return n, num_arvores_finitas

In [212]:
amst = []
for i in range(10000):
    restante_clientes, num_arvores_finitas = simulador(lambda_=4, mi_=2)
    amst.append(restante_clientes)

remaining_mean = mean(amst)
print(remaining_mean)

21.3997


In [215]:
#Cenários (λ, μ)
cenarios = [
    {'λ': 1, 'μ': 2},
    {'λ': 2, 'μ': 4},
    {'λ': 4, 'μ': 2},
]

for cenario in cenarios:
    print(f"----- Cenário: λ = {cenario['λ']}, μ = {cenario['μ']} -----")

    total = 100
    amostras = []    
    for i in range(total):
        inf, fin = 0, 0
        sub_total = 100
        for j in range(sub_total):
            restante_clientes, num_arvores_finitas = simulador(lambda_=cenario['λ'], mi_=cenario['μ'])
            if (num_arvores_finitas == 0):
                inf += 1
            elif (restante_clientes == 0):
                inf += 1
                
            fin += num_arvores_finitas
            amostras.append(fin/(fin + inf))

    print(f"Fração árvores finitas: {mean(amostras):.6f}")

----- Cenário: λ = 1, μ = 2 -----
Fração árvores finitas: 0.967196
----- Cenário: λ = 2, μ = 4 -----
Fração árvores finitas: 0.984107
----- Cenário: λ = 4, μ = 2 -----
Fração árvores finitas: 0.659735


In [70]:
import numpy as np
from collections import Counter

def mm1_analytical(ρ):
    L = ρ / (1 - ρ)
    Lq = ρ**2 / (1 - ρ)
    W = 1 / (2 - ρ)
    Wq = ρ / (2 * (2 - ρ))
    return {'L': L, 'Lq': Lq, 'W': W, 'Wq': Wq, 'ρ': ρ}

# Cenários (λ, μ e ρ)
cenarios = [
    {'λ': 1, 'μ': 2, 'ρ': 0.5},
    {'λ': 2, 'μ': 4, 'ρ': 0.5},
    {'λ': 4, 'μ': 2, 'ρ': 2},
]

for cenario in cenarios:
    print(f"----- Cenário: λ = {cenario['λ']}, μ = {cenario['μ']}, ρ = {cenario['ρ']} -----")
    
    result = simulador(lambda_=cenario['λ'], mi_=cenario['μ'], simulation_total_time=10000, deterministic=False)

    # Resultados da simulação
    # for key, value in result.items():
    #     print(f"{key}: {value}")

    # Proporção esperada de árvores finitas (de acordo com a heurística)
    proporcao_esperada = cenario['λ'] / (cenario['λ'] + cenario['μ'])

    # Proporção obtida na simulação
    proporcao_obtida = result['Proporção de Árvores Finitas']

    print(f"Proporção Esperada de Árvores Finitas: {proporcao_esperada:.6f}")
    print(f"Proporção Obtida na Simulação: {proporcao_obtida:.6f}")



----- Cenário: λ = 1, μ = 2, ρ = 0.5 -----
Proporção Esperada de Árvores Finitas: 0.333333
Proporção Obtida na Simulação: 0.180552
----- Cenário: λ = 2, μ = 4, ρ = 0.5 -----
Proporção Esperada de Árvores Finitas: 0.333333
Proporção Obtida na Simulação: 0.604859
----- Cenário: λ = 4, μ = 2, ρ = 2 -----
Proporção Esperada de Árvores Finitas: 0.666667
Proporção Obtida na Simulação: 0.000183


### Cenário 1:  $λ= 1, μ= 2, ρ= 0.5$

In [83]:
result = simulador(lambda_=12, mi_=10, simulation_total_time=10000, deterministic=False)
print(result['Proporção de Árvores Finitas'])

6.817964882935544e-05


### Cenário 2:  $λ= 2, μ= 4, ρ= 0.5$

In [52]:
result = simulador(lambda_=2, mi_=4, simulation_total_time=10000, deterministic=False)
print(result['Proporção de Árvores Finitas'])

0.6172327044025158


### Cenário 2:  $λ= 4, μ= 2, ρ= 2$

In [60]:
result = simulador(lambda_=4, mi_=2, simulation_total_time=10000, deterministic=False)
print(result['Proporção de Árvores Finitas'])

4.9896049896049896e-05


In [134]:
import numpy as np

# Poisson offspring distribution
def branch_poisson(n, lam):
    z = np.zeros(n + 1)
    z[0] = 1
    for i in range(1, n + 1):
        z[i] = np.sum(np.random.poisson(lam, int(z[i - 1])))
    return z

n = 5000
lambda_ = 4
mi_ = 2 
simlist = np.array([branch_poisson(10, lambda_ / mi_)[10] for _ in range(n)])
fraction_finite_trees = np.sum(simlist == 0) / n
print("Fração de árvores finitas:", fraction_finite_trees)

Fração de árvores finitas: 0.204


In [6]:
import numpy as np

def gaussian_elimination(A, B):
    n = len(B)
    
    # Combinação da matriz A com a matriz B
    augmented_matrix = np.hstack((A, B.reshape(n, 1)))

    # Fase de eliminação
    for i in range(n):
        # Pivô deve ser diferente de zero; se for, é necessário fazer um pivoteamento
        if augmented_matrix[i][i] == 0:
            for k in range(i + 1, n):
                if augmented_matrix[k][i] != 0:
                    augmented_matrix[[i, k]] = augmented_matrix[[k, i]]
                    break
        
        # Eliminação
        for j in range(i + 1, n):
            factor = augmented_matrix[j][i] / augmented_matrix[i][i]
            augmented_matrix[j] -= factor * augmented_matrix[i]

    # Fase de retrosubstituição
    solutions = np.zeros(n)
    for i in range(n - 1, -1, -1):
        solutions[i] = (augmented_matrix[i][-1] - np.dot(augmented_matrix[i][i+1:n], solutions[i+1:n])) / augmented_matrix[i][i]

    return solutions

# Exemplo de utilização:
A = np.array([[2, 3, -1],
              [1, -1, 1],
              [3, 2, 1]])

B = np.array([5, 2, 8])

solutions = gaussian_elimination(A, B)
print("Soluções:", solutions)


UFuncTypeError: Cannot cast ufunc 'subtract' output from dtype('float64') to dtype('int32') with casting rule 'same_kind'